Mestrando: Isac Lira <br>
Matrícula: 497776 <br>
Prof.: Guilherme Barreto

# Questão 1
(Classificação de Padrões) – Implementar os classificadores MQL (mínimos quadrados linear), Madaline com regra LMS padrão e logística, além da MLP (com 1 e 2 camadas ocultas)para o clássico problema de reconhecimentos de caracteres cursivos (handwritten digit) do MNIST.O banco de dados está disponível no sítio da internet abaixo
<br><br>

In [318]:
# Imports for deal with arrays
import numpy as np
import pandas as pd
import idx2numpy

# Imports for metrics and preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import StandardScaler

# Import classifiers 
from classifiers.mlp import MLP1, MLP2
from classifiers.adaline_classifier import ADALINEClassifier
from classifiers.linear_model import LeastSquareClassifier

# Import regression models
from regressors.least_square import LeastSquareRegressor
from regressors.adaline_regressor import ADALINERegressor
from regressors.mlp import build_model, train_model, predict

## Step 1:
Nas células a seguir é realizado o tratamento dos dados vetorizando as imagens e normalizando pela média e variância. 

In [260]:
# Read data obtained from http://yann.lecun.com/exdb/mnist/
train_images = idx2numpy.convert_from_file(
    'DigitData/train-images-idx3-ubyte.idx')
train_labels = idx2numpy.convert_from_file(
    'DigitData/train-labels-idx1-ubyte.idx')
test_images = idx2numpy.convert_from_file(
    'DigitData/t10k-images-idx3-ubyte.idx')
test_labels = idx2numpy.convert_from_file(
    'DigitData/t10k-labels-idx1-ubyte.idx')

In [265]:
# Reshape images to vectors
train_vetors = train_images.reshape(train_images.shape[0], -1)
test_vetors = test_images.reshape(test_images.shape[0], -1)

# Normalize vectors
norm_train_vetors = (train_vetors - np.mean(train_vetors))/np.std(train_vetors)
norm_test_vetors = (test_vetors - np.mean(train_vetors))/np.std(train_vetors)

In [266]:
X_train, y_train = norm_train_vetors, train_labels
X_test, y_test = norm_test_vetors, test_labels

## Step 2:
Após o tratamento dos dados, os modelos são treinados e avaliados usando conjunto de teste disponibilizado. O modelo MLP com uma cada oculta possui 100 neurônios e foi treinado por 20 épocas com uma taxa de aprendizagem de 0.1, já o modelo com 2 camadas ocultas possui 100 neurônios em cada camada e foi treinado por 50 épocas com a mesma taxa de aprendizagem anterior. 

O modelo ADALINE foi treinado com uma taxa de aprendizagem alpha de 0.001 durante 10 épocas. Todas as implementações estão em anexo.

In [268]:
# Set models

models = {
    'adaline_clf': ADALINEClassifier(),
    'ls_clf': LeastSquareClassifier(),
    'mlp1': MLP1(hidden_neurons=100,
              epoches=20,
              eta=0.1),
    'mlp2': MLP2(layers_size=(100, 100),
              epoches=50,
              eta=0.1)
}

results = {}
for model_name, clf in models.items():
    print(f'Training model {model_name}')    
    # Fit model
    clf.fit(X_train, y_train)
    # Compute accuray score on Test set
    results[model_name] = round(accuracy_score(y_test, clf.predict(X_test)), 2)

Training model adaline_clf
Training model ls_clf
Training model mlp1
Training model mlp2


In [309]:
# Create a dataframe with the results
df = pd.DataFrame(pd.Series(results), 
             columns=['Accuracy'])\
       .reset_index()\
       .rename(columns={'index': 'Model'})
df['Model']= df['Model'].map({
    'adaline_clf': 'ADALINE Classify',
    'ls_clf': 'Linear Least Square',
    'mlp1': 'MLP (1 hidden layer)',
    'mlp2': 'MLP (2 hidden layer)'
})

## Step 3: Resultados

In [304]:
# Perfomance on test set
df

,Model,Accuracy
0,ADALINE Classify,0.85
1,Linear Least Square,0.86
2,MLP (1 hidden layer),0.97
3,MLP (2 hidden layer),0.98


Podemos observar que os modelos MLP de 1 e 2 camadas ocultas alcançam peformances maiores que os modelos ADALINE e Linear Least Square, justificando sua maior complexidade.

# Questão 2

(Regressão e Ajuste de Curvas) – Implementar os modelos de regressão linear múltipla de mínimos quadrados, rede Adaline e rede MLP (1 e 2 camadas ocultas) ao seguinte conjunto de dados:
<br><br>

## Step 1: Processamento dos Dados
O dataset contém apenas 414 amostras e 7 variáveis dependentes. As variáveis `X1 transaction date` e `No` foram removidas do dataset e nenhuma etapa de engenharia de features foi realizada. Como etapa de preprocessamento, os dados foram normalizados pela média e variância.

In [315]:
# Read real state data
real_state_data = pd.read_excel("real_state_data2.xlsx")

In [335]:
real_state_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


In [316]:
# Clean the dataframe
data = real_state_data.drop(columns=['No'])
columns = list(data.columns)
map_cols = {c:c.split(' ')[0] for c in columns}

data = data.rename(columns=map_cols)

In [319]:
# Split in train and test
X, y = data.drop(['X1', 'Y'], axis=1).values, data['Y'].values
X_train, X_test, y_train, y_test = split(X, y, train_size=0.9)

# Normalize training data by mean and std
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Step 2: Treinamento dos Modelos

O modelo ADALINE foi treinado usando um passo `alpha` de 0.01 e 100 épocas de treinamento.
Os modelos MLP de regressão foram implementados usando Pytorch e estão disponibilizados em anexo. Para o modelo MLP de 1 camada oculta utilizou-se 200 neurônios e os pesos são otimizados usando `Adam` com 100 épocas e um batch size de 20 amostras. Para o modelo de 2 camadas, utilizou-se 100 neurônios em cada camada e 200 épocas treinamento.

Dividiu-se os dados em 90% para treino e 10% para teste onde o modelo foi avaliado computando o score R2.

In [320]:
# Train regressor models

model_labels = {}
models = {
    'ADALINE Regressor': ADALINERegressor(alpha=0.01, epoches=100),
    'Least Square Regressor': LeastSquareRegressor(),
    '1-NN': None,
    '2-NN': None,
}

results = {}
for model_name, regressor in models.items():
    print(f'Training model {model_name}')
    if model_name == '1-NN':
        
        mlp = build_model(n_feats=X_train.shape[1],
                          layers_size=[200])
        train_model(X=X_train,
                    y=y_train,
                    model=mlp,
                    epoches=100,
                    batch_size=20)

        results[model_name] = r2_score(y_test, predict(X_test, mlp))
        
    elif model_name == '2-NN':
        mlp = build_model(n_feats=X_train.shape[1],
                          layers_size=[100, 100])
        train_model(X=X_train,
                    y=y_train,
                    model=mlp,
                    epoches=200,
                    batch_size=20)

        results[model_name] = r2_score(y_test, predict(X_test, mlp))
        
    else:
        regressor.fit(X_train, y_train)
        results[model_name] = r2_score(y_test, regressor.predict(X_test))

Training model ADALINE Regressor
Training model Least Square Regressor
Training model 1-NN
Training model 2-NN


In [328]:
# Create a dataframe with the results
df = pd.DataFrame(pd.Series(results), 
             columns=['R2 Score'])\
       .reset_index()\
       .rename(columns={'index': 'Model'})
df

,Model,R2 Score
0,ADALINE Regressor,0.754030
1,Least Square Regressor,0.749548
2,1-NN,0.783041
3,2-NN,0.803900


Devido a maior complexidade do modelo MPL com 2 camadas, houve uma maior performance desse modelo para estimar os valores de teste.